In [2]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import math
from datetime import timedelta

In [3]:
##Convolution layer 1
filter_size1 = 5
num_filters1 = 16

##Convolutiona layer 2
filter_size2 = 5
num_filters2 = 36

##Fully Connected Layer
fc_size = 128

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

data = input_data.read_data_sets('data/MNIST',one_hot=True)

Extracting data/MNIST\train-images-idx3-ubyte.gz
Extracting data/MNIST\train-labels-idx1-ubyte.gz
Extracting data/MNIST\t10k-images-idx3-ubyte.gz
Extracting data/MNIST\t10k-labels-idx1-ubyte.gz


In [8]:
print('Size of')
print('Training Set\t\t{}'.format(len(data.train.labels)))
print('Test Set\t\t{}'.format(len(data.test.labels)))
print('Validation Set\t\t{}'.format(len(data.validation.labels)))

Size of
Training Set		55000
Test Set		10000
Validation Set		5000


In [102]:
data.test.cls = np.argmax(data.test.labels,axis=1)

In [103]:
data.test.cls[0:5]

array([7, 2, 1, 0, 4], dtype=int64)

In [18]:
##DIMENSIONS

img_size = 28

img_size_flat = img_size * img_size

img_shape = (img_size,img_size)

num_channels = 1

num_classes = 10

In [19]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.5))

In [20]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05,shape=[length])) ### 0.05 is the constant value for biases

In [37]:
def new_conv_layer(input,
                  num_input_channels,
                  filter_size,
                  nm_filters,
                  use_pooling=True):
    
    shape = [filter_size,filter_size,num_input_channels,nm_filters]
    
    weights = new_weights(shape)
    
    biases = new_biases(nm_filters)
    
    layer = tf.nn.conv2d(input = input,
                        filter = weights,
                        strides = [1,1,1,1],
                        padding = 'SAME')
    layer += biases
    
    if use_pooling:
        
        layer = tf.nn.max_pool(value = layer,
                              ksize = [1,2,2,1],
                              strides = [1,2,2,1],
                              padding = 'SAME')
        
    layer = tf.nn.relu(layer)
    
    return layer,weights

In [38]:
def flatten_layer(layer):
    
    layer_shape = layer.get_shape()
    
    num_features = layer_shape[1:4].num_elements()
    
    layer_flat = tf.reshape(layer,[-1,num_features])
    
    return layer_flat,num_features

In [51]:
def new_fc_layer(input,
                num_inputs,
                num_outputs,
                use_relu=True):
    
    weights = new_weights(shape=[num_inputs,num_outputs])
    
    biases = new_biases(length=num_outputs)
    
    layer = tf.matmul(input,weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)
        
    return layer


In [52]:
x = tf.placeholder(tf.float32,shape = [None,img_size_flat],name = 'x')

x_image = tf.reshape(x,[-1,img_size,img_size,num_channels])

y_true = tf.placeholder(tf.float32, shape = [None,num_classes],name = 'y_true')

y_true_cls = tf.argmax(y_true,axis=1)

In [53]:
layer_conv1, weights_conv1 = new_conv_layer(input = x_image,
                                           num_input_channels = num_channels,
                                           filter_size = filter_size1,
                                           nm_filters = num_filters1,
                                           use_pooling=True)

In [54]:
layer_conv2, weights_conv2 = new_conv_layer(input = layer_conv1,
                                           num_input_channels = num_filters1,
                                           filter_size = filter_size2,
                                           nm_filters = num_filters2,
                                           use_pooling=True)

In [55]:
layer_flat, num_features = flatten_layer(layer_conv2)

In [56]:
layer_fc1 = new_fc_layer(input = layer_flat,
                        num_inputs = num_features,
                        num_outputs = fc_size,
                        use_relu = True)

In [58]:
layer_fc2 = new_fc_layer(input = layer_fc1,
                        num_inputs = fc_size,
                        num_outputs = num_classes,
                        use_relu = False)

In [59]:
y_pred = tf.nn.softmax(layer_fc2)

y_pred_cls = tf.argmax(y_pred,axis = 1)

In [60]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)

cost = tf.reduce_mean(cross_entropy)

In [62]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls,y_true_cls) ## Calculates vector of booleans

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [64]:
session = tf.Session()

session.run(tf.global_variables_initializer())

In [65]:
train_batch_size = 64

In [72]:
total_iterations = 0

def optimize(num_iterations):
    global total_iterations

    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)

        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        session.run(optimizer, feed_dict=feed_dict_train)
        
        if i % 100 == 0:
    
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
            print(msg.format(i + 1, acc))

    total_iterations += num_iterations
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [73]:
def plot_example_errors(cls_pred, correct):
  
    incorrect = (correct == False)
    images = data.test.images[incorrect]
    
    cls_pred = cls_pred[incorrect]
    cls_true = data.test.cls[incorrect]
    
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

In [74]:
def plot_confusion_matrix(cls_pred):
 
    cls_true = data.test.cls
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    print(cm)
    plt.matshow(cm)
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

    plt.show()

In [75]:
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    num_test = len(data.test.images)

    cls_pred = np.zeros(shape=num_test, dtype=np.int)
    i = 0

    while i < num_test:
        j = min(i + test_batch_size, num_test)

        images = data.test.images[i:j, :]

        labels = data.test.labels[i:j, :]
        feed_dict = {x: images,
                     y_true: labels}

        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j
    cls_true = data.test.cls
    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test

    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [76]:
optimize(num_iterations=1000) 

Optimization Iteration:      1, Training Accuracy:   4.7%
Optimization Iteration:    101, Training Accuracy:  23.4%
Optimization Iteration:    201, Training Accuracy:  37.5%
Optimization Iteration:    301, Training Accuracy:  45.3%
Optimization Iteration:    401, Training Accuracy:  45.3%
Optimization Iteration:    501, Training Accuracy:  68.8%
Optimization Iteration:    601, Training Accuracy:  65.6%
Optimization Iteration:    701, Training Accuracy:  71.9%
Optimization Iteration:    801, Training Accuracy:  81.2%
Optimization Iteration:    901, Training Accuracy:  68.8%
Time usage: 0:02:15


In [78]:
optimize(num_iterations = 9000)

Optimization Iteration:   1001, Training Accuracy:  84.4%
Optimization Iteration:   1101, Training Accuracy:  78.1%
Optimization Iteration:   1201, Training Accuracy:  75.0%
Optimization Iteration:   1301, Training Accuracy:  82.8%
Optimization Iteration:   1401, Training Accuracy:  84.4%
Optimization Iteration:   1501, Training Accuracy:  73.4%
Optimization Iteration:   1601, Training Accuracy:  79.7%
Optimization Iteration:   1701, Training Accuracy:  82.8%
Optimization Iteration:   1801, Training Accuracy:  84.4%
Optimization Iteration:   1901, Training Accuracy:  84.4%
Optimization Iteration:   2001, Training Accuracy:  85.9%
Optimization Iteration:   2101, Training Accuracy:  87.5%
Optimization Iteration:   2201, Training Accuracy:  81.2%
Optimization Iteration:   2301, Training Accuracy:  90.6%
Optimization Iteration:   2401, Training Accuracy:  89.1%
Optimization Iteration:   2501, Training Accuracy:  82.8%
Optimization Iteration:   2601, Training Accuracy:  79.7%
Optimization I

In [108]:
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    num_test = len(data.test.images)

    cls_pred = np.zeros(shape=num_test, dtype=np.int)
    i = 0

    while i < num_test:
        j = min(i + test_batch_size, num_test)

        images = data.test.images[i:j, :]

        labels = data.test.labels[i:j, :]
        feed_dict = {x: images,
                     y_true: labels}

        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j
    cls_true = data.test.cls
    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test

    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [109]:
print_test_accuracy(show_example_errors=False,
                    show_confusion_matrix=False)

Accuracy on Test-Set: 95.0% (9498 / 10000)


In [110]:
optimize(num_iterations=100)

Optimization Iteration:  10001, Training Accuracy:  93.8%
Time usage: 0:00:14


In [111]:
print_test_accuracy()

Accuracy on Test-Set: 95.0% (9505 / 10000)


In [112]:
optimize(num_iterations=500)

Optimization Iteration:  10101, Training Accuracy:  96.9%
Optimization Iteration:  10201, Training Accuracy:  96.9%
Optimization Iteration:  10301, Training Accuracy:  96.9%
Optimization Iteration:  10401, Training Accuracy:  96.9%
Optimization Iteration:  10501, Training Accuracy:  93.8%
Time usage: 0:01:11


In [113]:
print_test_accuracy()

Accuracy on Test-Set: 95.0% (9503 / 10000)


In [114]:
optimize(num_iterations=1000)

Optimization Iteration:  10601, Training Accuracy:  95.3%
Optimization Iteration:  10701, Training Accuracy:  96.9%
Optimization Iteration:  10801, Training Accuracy:  93.8%
Optimization Iteration:  10901, Training Accuracy:  98.4%
Optimization Iteration:  11001, Training Accuracy: 100.0%
Optimization Iteration:  11101, Training Accuracy:  96.9%
Optimization Iteration:  11201, Training Accuracy:  95.3%
Optimization Iteration:  11301, Training Accuracy:  92.2%
Optimization Iteration:  11401, Training Accuracy:  96.9%
Optimization Iteration:  11501, Training Accuracy:  98.4%
Time usage: 0:02:16


In [115]:
print_test_accuracy()

Accuracy on Test-Set: 95.3% (9533 / 10000)
